# LAB 3: Creating a simple AI Agent with Langchain

## Initiation

In [ ]:
# Run this cell if you have not installed langchain in your environment
!pip install langchain
!pip install langchain-openai

In [ ]:
# loading environment variables 
from dotenv import load_dotenv
load_dotenv(override=True)  # take environment variables

In [ ]:
# Explain how to generate apikey from Tavily dashboard
TAVILY_API_KEY = "<insert your tavily key here>"

In [ ]:
# Initiating Langchain Chat Models
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4.1-mini", model_provider= "openai")

In [ ]:
# Since we need a tool from langchain community, we need to install the package first
!pip install langchain-community

## Tool Calling with Langchain

In [ ]:
# Tool 1: Google Search
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("langchain tutorial")

In [ ]:
# Tool 2: Weather
import requests
from langchain_core.tools import tool

@tool(parse_docstring=True)
def get_weather(latitude, longitude):
    """
        Use this tool to get current temperature and wind speed around a certain location. Search longitude and latitude from the internet.

        args:
            latitude : latitude value of the location
            longitude : longitude value of the location
    """
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']

In [ ]:
# Testing Tool 2
get_weather.invoke({'latitude': "-6.8999", "longitude": "108.89998"})

In [ ]:
!pip install yfinance

In [ ]:
# Tool 3: Stock Price 
import yfinance as yf

@tool(parse_docstring=True)
def get_stock_price(ticker: str) -> str:
    '''
        This function is called to get the latest closing stock price of a company. 
        
        Args:
            ticker (str): The stock ticker symbol (e.g., 'AAPL', 'GOOG').

        Returns:
            str: A message with the latest closing stock price, or an error message if data is unavailable.
    '''
    stock = yf.Ticker(ticker)
    data = stock.history(period="1d")
    if data.empty:
        return f"No data found for {ticker}."
    latest_price = data['Close'].iloc[-1]
    return f"The latest closing price of {ticker} is ${latest_price:.2f}"

In [ ]:
get_stock_price.invoke("AAPL")

In [ ]:
tools = [search, get_weather, get_stock_price]

## Building The AI Agent with Langchain

In [ ]:
model_with_tools = model.bind_tools(tools)

In [ ]:
# No Tool Calling
from langchain_core.messages import HumanMessage

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
# calling weather
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
from langchain.agents import create_tool_calling_agent
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(model, tools, prompt)

In [ ]:
prompt.pretty_print()

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
agent_executor.invoke({"input": "whats the weather in sf?"})

In [ ]:
agent_executor.invoke({"input": "Give me the stock price of Tesla"})

In [ ]:
# Calling two tools
for event in agent_executor.stream({"input": "what's the wind speed in Sumber?"}):
    print(event)

## END